# C++框架下的JUCE音频插件开发：从制作一个简单Filter开始
![]()
## 准备工作：
- 准备好IDE，VS或者Xcode都行，，这里用VisualStudio2022  
- 配置好C++的编译环境，这里用MSVC
- 准备好JUCE的开发库，可以使用Projucer或者Cmake来构建，Projucer由于有相对更加人性化的GUI界面所以更适合新人上手 

[JUCE的Github页面](https://github.com/juce-framework/JUCE)  
[JUCE的官网下载页面](https://juce.com/download/)  

## 配置环境：

在官网上下载Projucer，这是一个免安装的软件，解压到常用路径就可以了

![Projucer的下载](Image\DownloadProjucer.png)

打开可执行文件会弹出一个创建新工程的窗口，这里可以创建多种JUCE工程，既有独立运行的应用，也有各种格式的插件

![新建页面](Image\NewProject.png)

我们就以常用的VST3插件为例，不要忘记勾选juce_dsp的module，一会会用到

![创建VST](Image\CreateVST.png)

值得一提的是，Projucer也有管理解决方案文件的功能，所以一切有关文件层级的修改建议在Projucer中进行，否则可能会导致修改被覆盖  

例如这里我们可以在Projucer中添加额外的module，包括JUCE官方的module和用户自定义的module

![修改文件](Image\ModifyFile.png)

点击File选项卡下点击“Save Project and Open in IDE”

这样会自动唤起VS，可以看到解决方案资源管理器下已经有多个工程，分别是共享代码库，Standalone工程，VST3工程，以及VST3的清单文件

![资源管理器](Image\SolutionAndProject.png)

按F5运行，默认JUCE的构建设置是Standalone，Standalone会构建并执行一个能够独立运行的exe，能够方便地查看UI界面（生成文件也能在构建设置中修改）

但是由于这样并不能直观表现音频插件的DSP处理效果，所以我们还需要将插件插入别的程序中，

![默认生成的界面](Image\DefaultEditor.png)

右键VST的工程并生成，会在Bulid中的对应位置生成VST插件

![构建VST](Image\BuildVST.png)

JUCE的案例工程中自带一个轻量化的主机，可以识别并插入插件，也可以通过修改启动项来在我们的工程构建的过程中自动打开，

![]()

直接挂载在宿主上也是可以的，这里用的是Bitwig。可以看到现在空白的VST插件可以正常挂载

Bitwig的优势是即使在我们开发过程中插件崩溃，也不会影响宿主运行；而其他宿主可能在会和插件一起崩溃（点名Reaper）

![]()

## 代码部分：

回到我们的SharedCode，这里有外部依赖，以及一些需要用到的Module，这些Module目前都是JUCE帮我们配置的，后面也能自定义一些自己的Module

![]()

主要的功能是在工程名下的Source文件夹下实现，这里已经有4个文件，分别是PluginEditor和PluginProcessor以及各自的.cpp和.h文件

PluginEditor主要负责插件页面的设计以及UI交互功能，而PluginProcessor主要负责插件内部的逻辑，对于音频或者midi数据块的处理

![]()



当然想要创建更多脚本来管理也是可以的，别忘在Projucer中配置好

PluginProcessor中的类默认继承了juce::AudioProcessor中的类；PluginEditor中的类默认继承了juce::AudioEditor中的类，
所以这两个类都能override一些预先写好的方法

今天我们主要来看Processor中的内容，这是用来实现音频处理最核心的内容

![]()

.h和.cpp的分工比较明确，.h可以用来定义类和override一些方法，.cpp来实现方法

预先默认已经override了许多方法，但其实这些方法并不复杂，基本上功能都写在名字上了

![]()